# Pollution impact factors for photochemical ozone formation 

Exiobase categories:
- NMVOC – combustion – air
- NOx – combustion – air

LC-Impact stressors
- Terrestrial ecosystem damage
    - NOx
    - NMVOC

Note: calculate different stressor separately if you run out of memory

In [ ]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc
import json

# load arguments
with open("../arguments.json", "r") as f:
    arguments = json.load(f)

In [ ]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path=arguments["exio_11_path"])
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path=arguments["exio_19_path"])

## Calculate DRorigin
DRorigin is a matrix which describes the amount of the driver of biodiversity loss (DR) that occurs in impact region i sector k and is driven by consumption in region j sector k.

To calculate the matrix we need to
1. Aggregate relevant drivers from exiobase together
2. Diagonalize the aggregated driver and re-calculate the IO system

In [ ]:
# no need for aggregation

In [ ]:
# diagonalize both nox and nmvoc because CFs are found for both
#exio3_11.nox = exio3_11.satellite.copy()
exio3_11.nmvoc = exio3_11.satellite.copy()

# diagonalize and calculate land stress - annual and permanent 
nmvoc_diag = exio3_11.nmvoc.diag_stressor(("NMVOC - combustion - air"))
#nox_diag = exio3_11.nox.diag_stressor(("NOx - combustion - air"))
exio3_11.nmvoc_diag = nmvoc_diag
#exio3_11.nox_diag = nox_diag
exio3_11.calc_all()

In [ ]:
# diagonalize both nox and nmvoc because CFs are found for both
# (optimized diagonalization)
nmvoc_diag = exio3_11.satellite.diag_stressor(("NMVOC - combustion - air"))
nox_diag = exio3_11.satellite.diag_stressor(("NOx - combustion - air"))

# First, make sure that L matrix is calculated 
if exio3_11.L is None:
    # try loading the L matrix from pickles/exio3_11_L.pickle
    try:
        exio3_11.L = pd.read_pickle("pickles/exio3_11_L.pickle")
        print("L matrix loaded from pickle")
    except FileNotFoundError:
        print("L matrix not found, calculating it from scratch.")
        print("Calculating A")
        exio3_11.A = pymrio.calc_A(exio3_11.Z, exio3_11.x)
        print("Calculating L")
        exio3_11.L = pymrio.calc_L(exio3_11.A)
        # save the L matrix to a pickle file
        pd.to_pickle(exio3_11.L, "pickles/exio3_11_L.pickle")
else:
    print("L already loaded")

# calculate S (direct emission multipliers)
if nmvoc_diag.S is None:
    print("Calculating S")
    nmvoc_diag.S = pymrio.calc_S(nmvoc_diag.F, exio3_11.x)

if nox_diag.S is None:
    print("Calculating S")
    nox_diag.S = pymrio.calc_S(nox_diag.F, exio3_11.x)

Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()

nmvoc_diag.D_cba, _, _, _ = pymrio.calc_accounts(nmvoc_diag.S, exio3_11.L, Y_agg)
nox_diag.D_cba, _, _, _ = pymrio.calc_accounts(nox_diag.S, exio3_11.L, Y_agg)

In [5]:
nmvoc_diag.D_cba

region                                                           AT  \
sector                                                   Paddy rice   
region sector                                                         
AT     Paddy rice                                          0.000000   
       Wheat                                               0.000142   
       Cereal grains nec                                   0.000554   
       Vegetables, fruit, nuts                             0.000009   
       Oil seeds                                           0.000027   
...                                                             ...   
WM     Membership organisation services n.e.c. (91)        0.000077   
       Recreational, cultural and sporting services (92)   0.000244   
       Other services (93)                                 0.000249   
       Private households with employed persons (95)       0.000007   
       Extra-territorial organizations and bodies          0.000000   

region                                                                  \
sector                                                           Wheat   
region sector                                                            
AT     Paddy rice                                             0.000000   
       Wheat                                              13520.937887   
       Cereal grains nec                                      0.610740   
       Vegetables, fruit, nuts                                0.040506   
       Oil seeds                                              0.067525   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)           0.009829   
       Recreational, cultural and sporting services (92)      0.037207   
       Other services (93)                                    0.025825   
       Private households with employed persons (95)          0.001098   
       Extra-territorial organizations and bodies             0.000000   

region                                                                      \
sector                                                   Cereal grains nec   
region sector                                                                
AT     Paddy rice                                                 0.000000   
       Wheat                                                      1.856322   
       Cereal grains nec                                     106043.902703   
       Vegetables, fruit, nuts                                    0.237469   
       Oil seeds                                                  0.334599   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)               0.032473   
       Recreational, cultural and sporting services (92)          0.139985   
       Other services (93)                                        0.087703   
       Private households with employed persons (95)              0.003895   
       Extra-territorial organizations and bodies                 0.000000   

region                                                                            \
sector                                                   Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                       0.000000   
       Wheat                                                           16.084389   
       Cereal grains nec                                               35.093182   
       Vegetables, fruit, nuts                                     106769.634626   
       Oil seeds                                                        2.339032   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                     1.461714   
       Recreational, cultural and s

In [6]:
nox_diag.D_cba

region                                                           AT  \
sector                                                   Paddy rice   
region sector                                                         
AT     Paddy rice                                          0.000000   
       Wheat                                               0.000805   
       Cereal grains nec                                   0.002190   
       Vegetables, fruit, nuts                             0.000043   
       Oil seeds                                           0.000107   
...                                                             ...   
WM     Membership organisation services n.e.c. (91)        0.000123   
       Recreational, cultural and sporting services (92)   0.001550   
       Other services (93)                                 0.002810   
       Private households with employed persons (95)       0.000041   
       Extra-territorial organizations and bodies          0.000000   

region                                                                  \
sector                                                           Wheat   
region sector                                                            
AT     Paddy rice                                             0.000000   
       Wheat                                              76599.749768   
       Cereal grains nec                                      2.412708   
       Vegetables, fruit, nuts                                0.186634   
       Oil seeds                                              0.267258   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)           0.015747   
       Recreational, cultural and sporting services (92)      0.235903   
       Other services (93)                                    0.290965   
       Private households with employed persons (95)          0.006373   
       Extra-territorial organizations and bodies             0.000000   

region                                                                      \
sector                                                   Cereal grains nec   
region sector                                                                
AT     Paddy rice                                                 0.000000   
       Wheat                                                     10.516563   
       Cereal grains nec                                     418922.725710   
       Vegetables, fruit, nuts                                    1.094151   
       Oil seeds                                                  1.324312   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)               0.052027   
       Recreational, cultural and sporting services (92)          0.887541   
       Other services (93)                                        0.988117   
       Private households with employed persons (95)              0.022599   
       Extra-territorial organizations and bodies                 0.000000   

region                                                                            \
sector                                                   Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                       0.000000   
       Wheat                                                           91.122387   
       Cereal grains nec                                              138.634389   
       Vegetables, fruit, nuts                                     491946.113374   
       Oil seeds                                                        9.257675   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                     2.341876   
       Recreational, cultural and s

## Calculate DR share
DR share is a new matrix that represents the share of the driver in the impact region i from the total amount of driver that is driven by consumption in region j sector k.

To calculate the matrix each column of DR origin is shared by the sum of that column.

In [7]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns_nox = {}
for series_name, series in nox_diag.D_cba.items():
    series_sum = series.sum()
    columns_nox[series_name] = series / series_sum

dr_s_nox = pd.DataFrame(columns_nox)
dr_s_nox

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              2.344358e-07   
       Cereal grains nec                                  6.374609e-07   
       Vegetables, fruit, nuts                            1.265168e-08   
       Oil seeds                                          3.107563e-08   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       3.589327e-08   
       Recreational, cultural and sporting services (92)  4.511124e-07   
       Other services (93)                                8.180205e-07   
       Private households with employed persons (95)      1.181847e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              2.618657e-01   
       Cereal grains nec                                  8.248140e-06   
       Vegetables, fruit, nuts                            6.380320e-07   
       Oil seeds                                          9.136562e-07   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       5.383362e-08   
       Recreational, cultural and sporting services (92)  8.064633e-07   
       Other services (93)                                9.947005e-07   
       Private households with employed persons (95)      2.178709e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.314786e-05   
       Cereal grains nec                                      5.237391e-01   
       Vegetables, fruit, nuts                                1.367913e-06   
       Oil seeds                                              1.655661e-06   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           6.504425e-08   
       Recreational, cultural and sporting services (92)      1.109607e-06   
       Other services (93)                                    1.235349e-06   
       Private households with employed persons (95)          2.825305e-08   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        3.668645e-05   
       Cereal grains nec                                            5.581509e-05   
       Vegetables, fruit, nuts                                      1.980606e-01   
       Oil seeds                                                    3.727199e-06   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 9.428543e-07   
       Recreational, cultural and sporting services (92)     

In [8]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns_nmvoc = {}
for series_name, series in nmvoc_diag.D_cba.items():
    series_sum = series.sum()
    columns_nmvoc[series_name] = series / series_sum

dr_s_nmvoc = pd.DataFrame(columns_nmvoc)
dr_s_nmvoc

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              2.589616e-07   
       Cereal grains nec                                  1.009805e-06   
       Vegetables, fruit, nuts                            1.718345e-08   
       Oil seeds                                          4.913447e-08   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       1.401985e-07   
       Recreational, cultural and sporting services (92)  4.452550e-07   
       Other services (93)                                4.543608e-07   
       Private households with employed persons (95)      1.274698e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              3.160335e-01   
       Cereal grains nec                                  1.427522e-05   
       Vegetables, fruit, nuts                            9.467770e-07   
       Oil seeds                                          1.578309e-06   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       2.297349e-07   
       Recreational, cultural and sporting services (92)  8.696643e-07   
       Other services (93)                                6.036316e-07   
       Private households with employed persons (95)      2.567370e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.053256e-05   
       Cereal grains nec                                      6.016811e-01   
       Vegetables, fruit, nuts                                1.347374e-06   
       Oil seeds                                              1.898477e-06   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           1.842497e-07   
       Recreational, cultural and sporting services (92)      7.942567e-07   
       Other services (93)                                    4.976154e-07   
       Private households with employed persons (95)          2.209932e-08   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        3.578127e-05   
       Cereal grains nec                                            7.806817e-05   
       Vegetables, fruit, nuts                                      2.375193e-01   
       Oil seeds                                                    5.203402e-06   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 3.251723e-06   
       Recreational, cultural and sporting services (92)     

## Calculate DR unit
DR unit is a region harmonized version of DR share.

To calculate DR unit we need to:
1. Identify regions that are missing from exiobase (rest of the world regions) but are present in lc-impact.
2. Assume that the impacts are divided evenly across the rest of the world category
3. By that assumption we can add the missing countries to DR share as the impact of the row region where country belongs to divided by the number of countries in that region

In [56]:
# harmonize regions 

# for this harmonization we need to know which countries go into:
# 1. rest of the world europe
# 2. rest of the world asia pacific
# 3. rest of the world middle east
# 4. rest of the world africa
# 5. rest of the world americas

row_eu_countries = {
    'AL': 'Albania',
    'AZ': 'Azerbaijan',
    'BA': 'Bosnia and Herzegovina',
    'BY': 'Belarus',
    'IS': 'Iceland',
    'GL': 'Greenland',
    'GE': 'Georgia',
    'MK': 'Macedonia',
    'MD': 'Moldova',
    'RS': 'Serbia',
    'UA': 'Ukraine',
    # overseas territories of EU countries 
    'GP': 'Guadeloupe',
    'GF': 'French Guiana',
    'RE': 'Reunion',
    'VC': 'Saint Vincent and the Grenadines',
}

row_asia_pacific_countries = {
    'BD': 'Bangladesh', 'BN': 'Brunei', 'BT': 'Bhutan',
    'KH': 'Cambodia', 'FJ': 'Fiji', 'HK': 'Hong Kong',
    'KZ': 'Kazakhstan', 'KG': 'Kyrgyzstan', 'LA': 'Laos', 'MY': 'Malaysia', 'MV': 'Maldives',
    'MN': 'Mongolia', 'MM': 'Myanmar (Burma)', 'NP': 'Nepal', 'NZ': 'New Zealand',
    'KP': 'North Korea', 'PK': 'Pakistan', 'PG': 'Papua New Guinea', 'PH': 'Philippines',
    'WS': 'Samoa', 'SB': 'Solomon Islands', 'LK': 'Sri Lanka', 'SG': 'Singapore',
    'TJ': 'Tajikistan', 'TH': 'Thailand', 'TO': 'Tonga', 'TM': 'Turkmenistan',
    'UZ': 'Uzbekistan', 'VU': 'Vanuatu', 'VN': 'Vietnam'
}

row_african_countries = {
    'DZ': 'Algeria', 'AO': 'Angola', 'BJ': 'Benin', 'BW': 'Botswana', 'BF': 'Burkina Faso', 'BI': 'Burundi',
    'CM': 'Cameroon', 'CV': 'Cape Verde', 'CF': 'Central African Republic', 'TD': 'Chad', 'KM': 'Comoros',
    'CG': 'Congo', 'CD': 'Congo DRC', 'DJ': 'Djibouti', 'EG': 'Egypt', 'EH': 'Western Sahara', 'GQ': 'Equatorial Guinea', 'ER': 'Eritrea',
    'ET': 'Ethiopia', 'GA': 'Gabon', 'GM': 'Gambia, The', 'GH': 'Ghana', 'GN': 'Guinea', 'GW': 'Guinea-Bissau',
    'CI': 'Ivory Coast', 'KE': 'Kenya', 'LS': 'Lesotho', 'LR': 'Liberia', 'LY': 'Libya', 'MG': 'Madagascar',
    'MW': 'Malawi', 'ML': 'Mali', 'MR': 'Mauritania', 'MU': 'Mauritius', 'MA': 'Morocco', 'MZ': 'Mozambique', 'MQ': 'Martinique',
    'NA': 'Namibia', 'NE': 'Niger', 'NG': 'Nigeria', 'RW': 'Rwanda', 'ST': 'São Tomé and Príncipe', 'SN': 'Senegal',
    'SL': 'Sierra Leone', 'SO': 'Somalia', 'SZ': 'Eswatini',
    'SD': 'Sudan', 'TZ': 'Tanzania', 'TG': 'Togo', 'TN': 'Tunisia', 'UG': 'Uganda', 'ZM': 'Zambia', 'ZW': 'Zimbabwe'
}

row_american_countries = {
    'AW': 'Aruba', 'AR': 'Argentina', 'AN': 'Netherlands Antilles', 'BS': 'Bahamas, The', 'BZ': 'Belize', 'BO': 'Bolivia',
    'BB': 'Barbados', 'CL': 'Chile', 'CO': 'Colombia', 'CR': 'Costa Rica', 'CU': 'Cuba', 'DO': 'Dominican Republic',
    'EC': 'Ecuador', 'SV': 'El Salvador', 'GD': 'Grenada', 'GT': 'Guatemala', 'GY': 'Guyana', 'HT': 'Haiti', 'HN': 'Honduras', 'JM': 'Jamaica',
    'NI': 'Nicaragua', 'PA': 'Panama', 'PY': 'Paraguay', 'PE': 'Peru', 'PR': 'Puerto Rico', 'LC': 'Saint Lucia',
    'SR': 'Suriname', 'TT': 'Trinidad and Tobago', 'UY': 'Uruguay', 'VE': 'Venezuela'
}

row_middle_eastern_countries = {
    'AF': 'Afghanistan', 'AM': 'Armenia', 'BH': 'Bahrain',
    'IR': 'Iran', 'IQ': 'Iraq', 'IL': 'Israel', 'JO': 'Jordan', 'KW': 'Kuwait', 'LB': 'Lebanon', 'OM': 'Oman',
    'QA': 'Qatar', 'SA': 'Saudi Arabia', 'SY': 'Syria', 'AE': 'United Arab Emirates',
    'YE': 'Yemen'
}

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

# harmonize regions between lc-impact and exiobase
all_row_region_keys = list(row_eu_countries.keys()) + list(row_asia_pacific_countries.keys()) + list(row_african_countries.keys()) + list(row_american_countries.keys()) + list(row_middle_eastern_countries.keys())
exio_regions_without_row = [region for region in exio_regions if region not in row_regions.keys()]
print(len(exio_regions_without_row)) # exiobase contains 44 exact regions + 5 row regions

# combine exact regions from exiobase with exact row regions
all_regions = list(all_row_region_keys) + list(exio_regions_without_row)
print("Regions: ", len(all_regions))
print("Unique regions: ", len(set(all_regions)))
assert len(all_regions) == len(set(all_regions)), "There are duplicate regions in the all_regions list"

44
Regions:  187
Unique regions:  187


In [ ]:
# prepare lc-impact
lci = pd.read_excel(arguments["lc_impact_path"] + "/5-photochemical ozone formation/Photochemical_Ozone_formation.xlsx", sheet_name="CFs per country", skiprows=0, header=[0,1])
# Flatten the multi-level columns and rename them
lci.columns = [' '.join(col).strip() for col in lci.columns]
lci.rename(columns={lci.columns[0]: "Country", lci.columns[4]: "NOx", lci.columns[5]: "NMVOC"}, inplace=True)
lci.drop(columns=[lci.columns[1], lci.columns[2], lci.columns[3]], inplace=True)
lci

In [58]:
# Russia and Russia Europe have different CFs
# Russia Europe does not have ISO-3166 code
lci[lci["Country"].str.contains(r"Rus", regex=True)]
# drop Russia Europe from the LC-Impact data
lci = lci[lci["Country"] != "Russia Europe"].copy()

In [59]:
# Function to convert country name to ISO Alpha-2 code
def get_country_code(name):
    try:
        return pyc.countries.lookup(name).alpha_2
    except LookupError:
        try:
            return extra_mappings[name]
        except LookupError:
            print("Country code not found for ", name)
            return None  # Return None if country not found

extra_mappings = {
    "Turkey": "TR",
    "Russia": "RU",
    "Bahamas, The": "BS",
    "Byelarus": "BY",
    "Brunei": "BN",
    "Cape Verde": "CV",
    "China, Hong Kong Special Administrative Region": "HK",
    "Democratic Republic of the Congo": "CD",
    "Falkland Islands": "FK",
    "Gambia, The": "GM",
    "Ivory Coast": "CI",
    "Macedonia": "MK",
    "Myanmar (Burma)": "MM",
    "Netherlands Antilles": "AN",
    "Reunion": "RE",
    "Sao Tomo and Principe": "ST",
    "St. Vincent and the Grenadines": "VC",
    "Svalbard": "SJ",
    "Swaziland": "SZ",
    "US Virgin Islands": "VI",
    "Western Samoa": "WS",
}

lci["Country_Code"] = lci["Country"].apply(get_country_code)
# remove rows with missing country codes
lci = lci.dropna(subset=["Country_Code"])
lci.set_index("Country_Code", inplace=True)
lci

,Country,NOx,NMVOC
Country_Code,,,
AF,Afghanistan,2.900000e-15,1.100000e-16
AL,Albania,2.100000e-15,4.900000e-16
DZ,Algeria,4.900000e-15,7.700000e-16
AO,Angola,1.900000e-16,2.600000e-17
AR,Argentina,9.500000e-16,-1.800000e-17
...,...,...,...
VN,Vietnam,1.300000e-15,1.300000e-16
EH,Western Sahara,4.900000e-15,7.700000e-16
YE,Yemen,1.600000e-15,2.000000e-16


In [60]:
# check if lci regions has duplicates
duplicates = lci.index[lci.index.duplicated()]
print("Duplicates in lci regions: ", duplicates)
# CN is duplicated but values are the same, so we can drop one of them
lci.loc['CN']
lci = lci.loc[~lci.index.duplicated(keep='first')]

Duplicates in lci regions:  Index(['CN'], dtype='object', name='Country_Code')


In [61]:
# find if some regions are missing from the lci 
missing_regions = [region for region in all_regions if region not in lci.index.unique()]
print("Missing regions (lc-impact): ", missing_regions)

# find out if some regions are missing from augmented exiobase regions 
lci_regions = lci.index
missing_from_all_regions = [region for region in lci_regions if region not in all_regions]
print("Missing regions (exiobase, augmented): ", missing_from_all_regions)

Missing regions (lc-impact):  []
Missing regions (exiobase, augmented):  []


In [ ]:
# find if some regions are missing from the lci 
missing_regions = [region for region in all_regions if region not in lci.index.unique()]
print("Missing regions (lc-impact): ", missing_regions)

# find out if some regions are missing from augmented exiobase regions 
lci_regions = lci.index
missing_from_all_regions = [region for region in lci_regions if region not in all_regions]
print("Missing regions (exiobase, augmented): ", missing_from_all_regions)

# check if lci regions has duplicates
duplicates = lci.index[lci.index.duplicated()]
print("Duplicates in lci regions: ", duplicates)

# assert that all regions in lci-water are in the all_regions list
print(len(sorted(lci_regions)))
print(len(sorted(all_regions)))
assert sorted(lci_regions) == sorted(all_regions), "Some regions in lci are not in the all_regions list"

Missing regions (lc-impact):  []
Missing regions (exiobase, augmented):  []
Duplicates in lci regions:  Index([], dtype='object', name='Country_Code')
187
187


In [63]:
# augment dr_s to create dr_u

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl_nox = dr_s_nox.loc["WL"].copy()
wl_nox = wl_nox / len(row_american_countries)

we_nox = dr_s_nox.loc["WE"].copy()
we_nox = we_nox / len(row_eu_countries)

wa_nox = dr_s_nox.loc["WA"].copy()
wa_nox = wa_nox / len(row_asia_pacific_countries)

wf_nox = dr_s_nox.loc["WF"].copy()
wf_nox = wf_nox / len(row_african_countries)

wm_nox = dr_s_nox.loc["WM"].copy()
wm_nox = wm_nox / len(row_middle_eastern_countries)

dr_u_nox = dr_s_nox.copy()
dr_u_nox = dr_u_nox.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region_nox = {}
for region in all_row_region_keys:
    if region in row_eu_countries:
        country_to_region_nox[region] = we_nox
    elif region in row_asia_pacific_countries:
        country_to_region_nox[region] = wa_nox
    elif region in row_african_countries:
        country_to_region_nox[region] = wf_nox
    elif region in row_american_countries:
        country_to_region_nox[region] = wl_nox
    elif region in row_middle_eastern_countries:
        country_to_region_nox[region] = wm_nox
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u
all_indices = []
all_data = []
for region in all_row_region_keys:
    region_data = country_to_region_nox[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx_nox = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data_nox = pd.concat(all_data)
combined_data_nox.index = combined_idx_nox

dr_u_nox = pd.concat([dr_u_nox, combined_data_nox])
dr_u_nox

AT  \
                                                        Paddy rice   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              2.344358e-07   
   Cereal grains nec                                  6.374609e-07   
   Vegetables, fruit, nuts                            1.265168e-08   
   Oil seeds                                          3.107563e-08   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       2.392885e-09   
   Recreational, cultural and sporting services (92)  3.007416e-08   
   Other services (93)                                5.453470e-08   
   Private households with employed persons (95)      7.878981e-10   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                    \
                                                             Wheat   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              2.618657e-01   
   Cereal grains nec                                  8.248140e-06   
   Vegetables, fruit, nuts                            6.380320e-07   
   Oil seeds                                          9.136562e-07   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       3.588908e-09   
   Recreational, cultural and sporting services (92)  5.376422e-08   
   Other services (93)                                6.631337e-08   
   Private households with employed persons (95)      1.452473e-09   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.314786e-05   
   Cereal grains nec                                      5.237391e-01   
   Vegetables, fruit, nuts                                1.367913e-06   
   Oil seeds                                              1.655661e-06   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           4.336283e-09   
   Recreational, cultural and sporting services (92)      7.397382e-08   
   Other services (93)                                    8.235660e-08   
   Private households with employed persons (95)          1.883536e-09   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        3.668645e-05   
   Cereal grains nec                                            5.581509e-05   
   Vegetables, fruit, nuts                                      1.980606e-01   
   Oil seeds                                                    3.727199e-06   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 6.285696e-08   
   Recreational, cultural and sporting services (92)            2.500312e-07   
   Other services (93)                                          5.458023e-07   
   Private households with employed persons (95)                4.862843e-09   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                    

In [64]:
# augment dr_s to create dr_u

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl = dr_s_nmvoc.loc["WL"].copy()
wl = wl / len(row_american_countries)

we = dr_s_nmvoc.loc["WE"].copy()
we = we / len(row_eu_countries)

wa = dr_s_nmvoc.loc["WA"].copy()
wa = wa / len(row_asia_pacific_countries)

wf = dr_s_nmvoc.loc["WF"].copy()
wf = wf / len(row_african_countries)

wm = dr_s_nmvoc.loc["WM"].copy()
wm = wm / len(row_middle_eastern_countries)

dr_u_nmvoc = dr_s_nmvoc.copy()
dr_u_nmvoc = dr_u_nmvoc.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in all_row_region_keys:
    if region in row_eu_countries:
        country_to_region[region] = we
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa
    elif region in row_african_countries:
        country_to_region[region] = wf
    elif region in row_american_countries:
        country_to_region[region] = wl
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u
all_indices = []
all_data = []
for region in all_row_region_keys:
    region_data = country_to_region[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u_nmvoc = pd.concat([dr_u_nmvoc, combined_data])
dr_u_nmvoc

AT  \
                                                        Paddy rice   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              2.589616e-07   
   Cereal grains nec                                  1.009805e-06   
   Vegetables, fruit, nuts                            1.718345e-08   
   Oil seeds                                          4.913447e-08   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       9.346567e-09   
   Recreational, cultural and sporting services (92)  2.968367e-08   
   Other services (93)                                3.029072e-08   
   Private households with employed persons (95)      8.497990e-10   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                    \
                                                             Wheat   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              3.160335e-01   
   Cereal grains nec                                  1.427522e-05   
   Vegetables, fruit, nuts                            9.467770e-07   
   Oil seeds                                          1.578309e-06   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       1.531566e-08   
   Recreational, cultural and sporting services (92)  5.797762e-08   
   Other services (93)                                4.024211e-08   
   Private households with employed persons (95)      1.711580e-09   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.053256e-05   
   Cereal grains nec                                      6.016811e-01   
   Vegetables, fruit, nuts                                1.347374e-06   
   Oil seeds                                              1.898477e-06   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           1.228331e-08   
   Recreational, cultural and sporting services (92)      5.295044e-08   
   Other services (93)                                    3.317436e-08   
   Private households with employed persons (95)          1.473288e-09   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        3.578127e-05   
   Cereal grains nec                                            7.806817e-05   
   Vegetables, fruit, nuts                                      2.375193e-01   
   Oil seeds                                                    5.203402e-06   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 2.167816e-07   
   Recreational, cultural and sporting services (92)            2.178998e-07   
   Other services (93)                                          2.676767e-07   
   Private households with employed persons (95)                4.631002e-09   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                    

## Calculate DR factor
Calculate the impact factors of the driver for each impact region i driven by consumption in region j product sector k.

These impact factors tell how

To calculate DR factor:
1. Calculate the monetary impact factor impact/€ from 2019 exiobase data
2. Multiply each column of DR unit with the impact factors of consumption region j in product sector k. The resulting matrix represents the distribution of impacts of 1€ consumption in each consumption region.

In [65]:
# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k

dr_f_nox = dr_u_nox.copy()
total_nox = exio3_19.satellite.M.loc["NOx - combustion - air"]
scalars = total_nox.to_numpy() # multipliers for each column
print(len(scalars))
print(dr_u_nox.shape) 

# multiply each column of dr_u by the respective column value from exio3_19 impact factors
dr_f_nox = dr_f_nox * scalars # same as dr_f * diag(scalars) but more efficient with numpy broadcasting
dr_f_nox

9800
(37400, 9800)


AT               \
                                                     Paddy rice        Wheat   
AT Paddy rice                                               0.0     0.000000   
   Wheat                                                    0.0  1411.362074   
   Cereal grains nec                                        0.0     0.044455   
   Vegetables, fruit, nuts                                  0.0     0.003439   
   Oil seeds                                                0.0     0.004924   
...                                                         ...          ...   
YE Membership organisation services n.e.c. (91)             0.0     0.000019   
   Recreational, cultural and sporting services (92)        0.0     0.000290   
   Other services (93)                                      0.0     0.000357   
   Private households with employed persons (95)            0.0     0.000008   
   Extra-territorial organizations and bodies               0.0     0.000000   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                                 0.000000   
   Wheat                                                      0.040338   
   Cereal grains nec                                       1606.838022   
   Vegetables, fruit, nuts                                    0.004197   
   Oil seeds                                                  0.005080   
...                                                                ...   
YE Membership organisation services n.e.c. (91)               0.000013   
   Recreational, cultural and sporting services (92)          0.000227   
   Other services (93)                                        0.000253   
   Private households with employed persons (95)              0.000006   
   Extra-territorial organizations and bodies                 0.000000   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                       0.000000   
   Wheat                                                            0.034714   
   Cereal grains nec                                                0.052814   
   Vegetables, fruit, nuts                                        187.409698   
   Oil seeds                                                        0.003527   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                     0.000059   
   Recreational, cultural and sporting services (92)                0.000237   
   Other services (93)                                              0.000516   
   Private households with employed persons (95)                    0.000005   
   Extra-territorial organizations and bodies                       0.000000   

                                                                 \
                                                      Oil seeds   
AT Paddy rice                                          0.000000   
   Wheat                                               0.007889   
   Cereal grains nec                                   0.008523   
   Vegetables, fruit, nuts                             0.000756   
   Oil seeds                                          17.531564   
...                                                         ...   
YE Membership organisation services n.e.c. (91)        0.000024   
   Recreational, cultural and sporting services (92)   0.000179   
   Other services (93)                                 0.000265   
   Private households with employed persons (95)       0.000008   
   Extra-territorial organizations and bodies          0.000000   

                                                                             \
                                                     Sugar cane, 

In [66]:
# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k

dr_f_nmvoc = dr_u_nmvoc.copy()
total_nmvoc = exio3_19.satellite.M.loc["NMVOC - combustion - air"]
scalars = total_nmvoc.to_numpy() # multipliers for each column
print(len(scalars))
print(dr_u_nmvoc.shape) 

# multiply each column of dr_u by the respective column value from exio3_19 impact factors
dr_f_nmvoc = dr_f_nmvoc * scalars # same as dr_f * diag(scalars) but more efficient with numpy broadcasting
dr_f_nmvoc

9800
(37400, 9800)


AT              \
                                                     Paddy rice       Wheat   
AT Paddy rice                                               0.0    0.000000   
   Wheat                                                    0.0  284.810755   
   Cereal grains nec                                        0.0    0.012865   
   Vegetables, fruit, nuts                                  0.0    0.000853   
   Oil seeds                                                0.0    0.001422   
...                                                         ...         ...   
YE Membership organisation services n.e.c. (91)             0.0    0.000014   
   Recreational, cultural and sporting services (92)        0.0    0.000052   
   Other services (93)                                      0.0    0.000036   
   Private households with employed persons (95)            0.0    0.000002   
   Extra-territorial organizations and bodies               0.0    0.000000   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                                 0.000000   
   Wheat                                                      0.007630   
   Cereal grains nec                                        435.866083   
   Vegetables, fruit, nuts                                    0.000976   
   Oil seeds                                                  0.001375   
...                                                                ...   
YE Membership organisation services n.e.c. (91)               0.000009   
   Recreational, cultural and sporting services (92)          0.000038   
   Other services (93)                                        0.000024   
   Private households with employed persons (95)              0.000001   
   Extra-territorial organizations and bodies                 0.000000   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        6.088384e-03   
   Cereal grains nec                                            1.328374e-02   
   Vegetables, fruit, nuts                                      4.041525e+01   
   Oil seeds                                                    8.853881e-04   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 3.688660e-05   
   Recreational, cultural and sporting services (92)            3.707687e-05   
   Other services (93)                                          4.554670e-05   
   Private households with employed persons (95)                7.879910e-07   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                \
                                                     Oil seeds   
AT Paddy rice                                         0.000000   
   Wheat                                              0.001428   
   Cereal grains nec                                  0.002213   
   Vegetables, fruit, nuts                            0.000168   
   Oil seeds                                          4.543633   
...                                                        ...   
YE Membership organisation services n.e.c. (91)       0.000015   
   Recreational, cultural and sporting services (92)  0.000029   
   Other services (93)                                0.000024   
   Private households with employed persons (95)      0.000001   
   Extra-territorial organizations and bodies         0.000000   

                                                                             \
                                                     Sugar cane, sugar beet   
AT Paddy ric

## Calculate BDe factors (PDF/€ for each consumption region and product sector)

1. Multiply each cell of dr_u with the CF (characterisation factor from lc-impact) of the impact region to get PDF values for every entry
2. Sum up columns to get the total PDF/€ for the consumption region j product sector k

In [67]:
# sort rows on lci in same order as dr_f.index.sortlevel
lci = lci.reindex(dr_f_nox.index.get_level_values(0).unique())

# build array from the relevent lci stressor
# every value should be repeated 200 times (number of sectors)
cf_nox = lci["NOx"].to_numpy()
cf_nox = np.repeat(cf_nox, 200) # 1D array of length 200 * number of regions in lci
print(cf_nox.shape)
# expand cf_all_effects to match the shape of dr_f
cf_nox = np.tile(cf_nox, (dr_f_nox.shape[1], 1)).T
print(cf_nox.shape)

pdf = dr_f_nox * cf_nox
pdf

(37400,)
(37400, 9800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  1.411362e-12   
   Cereal grains nec                                        0.0  4.445451e-17   
   Vegetables, fruit, nuts                                  0.0  3.438763e-18   
   Oil seeds                                                0.0  4.924279e-18   
...                                                         ...           ...   
YE Membership organisation services n.e.c. (91)             0.0  3.094868e-20   
   Recreational, cultural and sporting services (92)        0.0  4.636317e-19   
   Other services (93)                                      0.0  5.718484e-19   
   Private households with employed persons (95)            0.0  1.252529e-20   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  4.033778e-17   
   Cereal grains nec                                      1.606838e-12   
   Vegetables, fruit, nuts                                4.196772e-18   
   Oil seeds                                              5.079589e-18   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           2.128603e-20   
   Recreational, cultural and sporting services (92)      3.631241e-19   
   Other services (93)                                    4.042737e-19   
   Private households with employed persons (95)          9.245940e-21   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        3.471360e-17   
   Cereal grains nec                                            5.281357e-17   
   Vegetables, fruit, nuts                                      1.874097e-13   
   Oil seeds                                                    3.526764e-18   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 9.516280e-20   
   Recreational, cultural and sporting services (92)            3.785367e-19   
   Other services (93)                                          8.263219e-19   
   Private households with employed persons (95)                7.362141e-21   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              7.889003e-18   
   Cereal grains nec                                  8.523165e-18   
   Vegetables, fruit, nuts                            7.557493e-19   
   Oil seeds                                          1.753156e-14   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       3.870641e-20   
   Recreational, cultural and sporting services (92)  2.867655e-19   
   Other services (93)                                4.238362e-19   
   Private households with employed persons (95)      1.222303e-20   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

In [68]:
# sum up all columns to get the total impact for each region-sector pair
pdf_total_nox = pdf.sum()
pdf_total_nox

AT  Paddy rice                                           0.000000e+00
    Wheat                                                4.851067e-12
    Cereal grains nec                                    3.085262e-12
    Vegetables, fruit, nuts                              1.527652e-12
    Oil seeds                                            2.092851e-12
                                                             ...     
WM  Membership organisation services n.e.c. (91)         1.922211e-12
    Recreational, cultural and sporting services (92)    2.042471e-12
    Other services (93)                                  2.417293e-12
    Private households with employed persons (95)        9.492392e-13
    Extra-territorial organizations and bodies           0.000000e+00
Length: 9800, dtype: float64

In [69]:
pd.DataFrame(pdf_total_nox).to_csv("csv/pdf-pollution-ozone-nox.csv", index=True)

In [70]:
# sort rows on lci in same order as dr_f.index.sortlevel
lci = lci.reindex(dr_f_nmvoc.index.get_level_values(0).unique())

# build array from the relevent lci stressor
# every value should be repeated 200 times (number of sectors)
cf_nmvoc = lci["NMVOC"].to_numpy()
cf_nmvoc = np.repeat(cf_nmvoc, 200) # 1D array of length 200 * number of regions in lci
print(cf_nmvoc.shape)
# expand cf_all_effects to match the shape of dr_f
cf_nmvoc = np.tile(cf_nmvoc, (dr_f_nmvoc.shape[1], 1)).T
print(cf_nmvoc.shape)

pdf = dr_f_nmvoc * cf_nmvoc
pdf

(37400,)
(37400, 9800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  2.193043e-13   
   Cereal grains nec                                        0.0  9.905966e-18   
   Vegetables, fruit, nuts                                  0.0  6.569944e-19   
   Oil seeds                                                0.0  1.095232e-18   
...                                                         ...           ...   
YE Membership organisation services n.e.c. (91)             0.0  2.760508e-21   
   Recreational, cultural and sporting services (92)        0.0  1.044994e-20   
   Other services (93)                                      0.0  7.253272e-21   
   Private households with employed persons (95)            0.0  3.084966e-22   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  5.875048e-18   
   Cereal grains nec                                      3.356169e-13   
   Vegetables, fruit, nuts                                7.515633e-19   
   Oil seeds                                              1.058968e-18   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           1.779640e-21   
   Recreational, cultural and sporting services (92)      7.671607e-21   
   Other services (93)                                    4.806393e-21   
   Private households with employed persons (95)          2.134541e-22   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        4.688056e-18   
   Cereal grains nec                                            1.022848e-17   
   Vegetables, fruit, nuts                                      3.111974e-14   
   Oil seeds                                                    6.817489e-19   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 7.377320e-21   
   Recreational, cultural and sporting services (92)            7.415373e-21   
   Other services (93)                                          9.109339e-21   
   Private households with employed persons (95)                1.575982e-22   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              1.099868e-18   
   Cereal grains nec                                  1.704087e-18   
   Vegetables, fruit, nuts                            1.295528e-19   
   Oil seeds                                          3.498597e-15   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       3.097702e-21   
   Recreational, cultural and sporting services (92)  5.799322e-21   
   Other services (93)                                4.823486e-21   
   Private households with employed persons (95)      2.701166e-22   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

In [71]:
# sum up all columns to get the total impact for each region-sector pair
pdf_total_nmvoc = pdf.sum()
pdf_total_nmvoc

AT  Paddy rice                                           0.000000e+00
    Wheat                                                5.655480e-13
    Cereal grains nec                                    5.227295e-13
    Vegetables, fruit, nuts                              1.072359e-13
    Oil seeds                                            5.096102e-13
                                                             ...     
WM  Membership organisation services n.e.c. (91)         1.110353e-13
    Recreational, cultural and sporting services (92)    7.449477e-14
    Other services (93)                                  8.617463e-14
    Private households with employed persons (95)        4.057350e-14
    Extra-territorial organizations and bodies           0.000000e+00
Length: 9800, dtype: float64

In [72]:
pd.DataFrame(pdf_total_nmvoc).to_csv("csv/pdf-pollution-ozone-nmvoc.csv", index=True)